# RCU Metrics Tests

In [21]:
%load_ext autoreload
%autoreload 2
import os
os.chdir("/scratch/ewalt/pdm/rs-uncertainty")
from src.rcu_metrics import StratifiedRCUSubset, StratifiedRCU, nan_frac, weighted_avg
import rasterio
import argparse
import yaml
import numpy as np
from tqdm import tqdm
from pathlib import Path
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def pjoin(*subs): return Path(os.path.abspath(os.path.join(*subs)))
def get_variance_bounds(cfg, N=2):
    for k, v in cfg.items():
        if k.endswith("_dir"): cfg[k] = Path(v)
    projects = cfg["projects_east"]+cfg["projects_west"]+cfg["projects_north"]
    # loop on projects to get variance bounds
    print(f"Computing variance bounds in {cfg['pkl_dir']}...")
    lo_variance = np.full((5,), np.inf)
    hi_variance = np.full((5,), -np.inf)
    variance_files = []
    # for variance_file in tqdm(list(cfg["prediction_dir"].glob("*_variance.tif"))):
    for variance_file in tqdm(list(cfg["prediction_dir"].glob("*_variance.tif"))[:N]): # Debug
        if variance_file.stem.split("_")[0] not in projects: continue
        variance_files.append(variance_file)
        with rasterio.open(variance_file) as fh:
            variance = fh.read(fh.indexes)
        variance_flat = variance.reshape(5, -1)
        hi = np.nanmax(variance_flat, axis=1)
        lo = np.nanmin(variance_flat, axis=1)
        hi_variance[hi>hi_variance] = hi[hi>hi_variance]
        lo_variance[lo<lo_variance] = lo[lo<lo_variance]
    print("Variances lower bound:", lo_variance.tolist())
    print("Variances upper bound:", hi_variance.tolist())
    # initialize RCU metrics
    return lo_variance, hi_variance, variance_files
def get_projects(cfg, variance_files):
    projects = []
    # Load standardization data
    with pjoin(cfg["pkl_dir"], "stats.yaml").open("r", encoding="utf-8") as f:
        stats = yaml.safe_load(f)
    labels_mean = np.array(stats["labels_stats"]["mean"]).reshape(5,1,1)
    labels_std = np.array(stats["labels_stats"]["std"]).reshape(5,1,1)
    # compute stats online
    for variance_file in tqdm(variance_files):
        # load data
        project = variance_file.stem.split('_')[0]
        # if project not in projects: continue
        with rasterio.open(pjoin(cfg['prediction_dir'], f"{project}_mean.tif")) as fh:
            mean = fh.read(fh.indexes)
        with rasterio.open(variance_file) as fh:
            variance = fh.read(fh.indexes)
        with rasterio.open(pjoin(cfg['gt_dir'], f"{project}.tif")) as fh:
            gt = fh.read(fh.indexes)
            gt[2] /= 100 # Cover/Dens normalization!!
            gt[4] /= 100
        # standardize
        mean[[0,1]] = (mean[[0,1]]-labels_mean[[0,1]])/labels_std[[0,1]]
        gt[[0,1]] = (gt[[0,1]]-labels_mean[[0,1]])/labels_std[[0,1]]
        projects.append((project, gt, mean, variance))
    return projects    
def res2df(res, cfg):
    R = {}
    for eid, eres in res.items():
        for metric_name, metric_info in eres.items():
            R[(eid, metric_name)] = metric_info["values"]
            R[(eid, f"ause-{metric_name}")] = metric_info["ause"]
    return pd.DataFrame(R, index=cfg["variable_names"]).T
def test(metrics, lo_variance, hi_variance, projects, cfg, N=2):
    print("creating")
    rcu = rcu = StratifiedRCUSubset(
        metric_names=metrics,
        num_variables=len(cfg["data_bands"]),
        # num_groups=len(projects),
        num_groups=N, # Debug
        num_bins=cfg["num_bins"],
        lo_variance=lo_variance,
        hi_variance=hi_variance
    )
    print("adding")
    results = {}
    for i, project in enumerate(projects):
        print("adding", project[0])
        rcu.add_project(*project)
        results[project[0]] = rcu.get_subset([project[0]])
    results["global"] = rcu.get()
    return res2df(results, cfg), rcu

In [4]:
with Path("./config/evaluate_testset/baseline.yaml").open("r", encoding="utf-8") as f:
    cfg = yaml.safe_load(f)
lo_variance, hi_variance, variance_files = get_variance_bounds(cfg)
projects = get_projects(cfg, variance_files)

Computing variance bounds in data/2023-04-05_18-58-33...


100%|████████████████████████████████████████| 2/2 [00:02<00:00,  1.39it/s]


Variances lower bound: [0.4428499639034271, 0.08763699978590012, 0.0004826262593269348, 0.0001668027980485931, 4.102319962839829e-06]
Variances upper bound: [268.0447998046875, 106.31964111328125, 0.11541682481765747, 0.0358952134847641, 0.18080325424671173]


100%|████████████████████████████████████████| 2/2 [00:08<00:00,  3.75s/it]


### Error Metrics [pOK]

In [5]:
res, rcu = test(["mse", "rmse", "mae", "mbe"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:589] (5, 1000)
[debug:600] (5, 1000)
[debug:589] (5, 1000)
[debug:600] (5, 1000)
[debug:589] (5, 1000)
[debug:600] (5, 1000)
[debug:589] (5, 1000)
[debug:600] (5, 1000)
[debug:546] Getting: <src.rcu_metrics.StratifiedMSE object at 0x7fccf7551c10>
[debug:213] getting [0]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.305 ((1, 1000))


/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:31: RuntimeWarning: invalid value encountered in true_divide
  return np.nansum(values*counts, axis=axis, keepdims=keepdims)/np.nansum(counts, axis=axis, keepdims=keepdims)


[debug:227] @get, reskeys: dict_keys([])
[debug:239] @get_subset, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedRMSE object at 0x7fccf7569250>
[debug:213] getting [0]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.305 ((1, 1000))
[debug:227] @get, reskeys: dict_keys([])
[debug:239] @get_subset, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedMAE object at 0x7fccf7569590>
[debug:213] getting [0]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.305 ((1, 1000))
[debug:227] @get, reskeys: dict_keys([])
[debug:239] @get_subset, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedMBE object at 0x7fccf7569090>
[debug:213] getting [0]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.305 ((1, 1000))
[debug:227] @get, reskeys: dict_keys([])
[debug:239] @get_subset, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
adding 4

In [6]:
res

P95     MeanH      Dens      Gini     Cover
439    mse        0.325619  0.345869  0.014802  0.002589  0.019557
       ause-mse   0.316299  0.332808  0.013426  0.002458  0.017563
       rmse       0.546519  0.566615  0.115757  0.048444  0.127427
       ause-rmse  0.537798  0.554638  0.110258  0.047084  0.120157
       mae        0.435301  0.450296  0.092023  0.037767  0.101679
       ause-mae   0.428070  0.440366  0.087426  0.036710  0.095359
       mbe       -0.042112 -0.027191 -0.004988 -0.003574 -0.015277
       ause-mbe  -0.042008 -0.026560 -0.003587 -0.003595 -0.013693
471    mse        0.267499  0.256655  0.014136  0.002227  0.022606
       ause-mse   0.261826  0.248938  0.013323  0.002123  0.021195
       rmse       0.492878  0.475826  0.113211  0.044448  0.142648
       ause-rmse  0.487471  0.468161  0.109352  0.043481  0.136706
       mae        0.387137  0.376022  0.089598  0.034760  0.113103
       ause-mae   0.382698  0.369817  0.086444  0.034003  0.108049
       mbe        0.033394  0.029016  0.000894  0.000052  0.006670
       ause-mbe   0.032833  0.028638  0.000985  0.000020  0.006062
global mse        0.275376  0.268746  0.014226  0.002276  0.022193
       ause-mse   0.283445  0.276293  0.013825  0.002284  0.020663
       rmse       0.500148  0.488131  0.113556  0.044989  0.140585
       ause-rmse  0.506596  0.494519  0.111485  0.045093  0.133984
       mae        0.393665  0.386088  0.089927  0.035168  0.111555
       ause-mae   0.398591  0.391233  0.088243  0.035224  0.105927
       mbe        0.023160  0.021398  0.000097 -0.000439  0.003696
       ause-mbe   0.025394  0.023076  0.000251 -0.000388  0.003514

### NLL [pOK]

In [7]:
res, rcu = test(["nll"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:589] (5, 1000)
[debug:600] (5, 1000)
[debug:546] Getting: <src.rcu_metrics.StratifiedNLL object at 0x7fccf5fd6650>
[debug:213] getting [0]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.305 ((1, 1000))
[debug:227] @get, reskeys: dict_keys([])
[debug:239] @get_subset, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
adding 471


/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:31: RuntimeWarning: invalid value encountered in true_divide
  return np.nansum(values*counts, axis=axis, keepdims=keepdims)/np.nansum(counts, axis=axis, keepdims=keepdims)


[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:589] (5, 1000)
[debug:600] (5, 1000)
[debug:546] Getting: <src.rcu_metrics.StratifiedNLL object at 0x7fccf5fd6650>
[debug:213] getting [1]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.438 ((1, 1000))
[debug:227] @get, reskeys: dict_keys(['__0__'])
[debug:239] @get_subset, reskeys: dict_keys(['__0__', '__1__'])
[debug:624] (5,) (5,)
[debug:217] globalres? True
[debug:184] Arr[2] nan: 0.3715 ((2, 1000))
[debug:227] @get, reskeys: dict_keys(['__0__', '__1__', '__all__'])


In [8]:
res

P95     MeanH      Dens      Gini     Cover
439    nll       0.962645  0.672108 -1.636143 -2.537406 -1.618837
       ause-nll  0.946107  0.648937 -1.683600 -2.545449 -1.685208
471    nll       0.742455  0.361909 -1.559775 -2.579001 -1.369920
       ause-nll  0.731254  0.345214 -1.599821 -2.594327 -1.425450
global nll       0.772297  0.403950 -1.570125 -2.573364 -1.403655
       ause-nll  0.782579  0.417078 -1.588992 -2.563738 -1.468072

### UCE, ENCE [pOK]

In [9]:
res, rcu = test(["uce", "ence"], lo_variance, hi_variance, [projects[1], projects[0]], cfg)

creating
adding
adding 471
[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:587] [(5, 1000), (5, 1000)]
[debug:591] 0, 0.5102, 0.0
[debug:587] [(5, 1000), (5, 1000)]
[debug:591] 0, 0.5102, 0.0
[debug:546] Getting: <src.rcu_metrics.StratifiedUCE object at 0x7fccf5fcb850>
[debug:280] globalres? False
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:293] @get, reskeys: dict_keys([])
[debug:624] (5,) (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedENCE object at 0x7fccf5fcb790>
[debug:280] globalres? False
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:293] @get, reskeys: dict_keys([])
[debug:624] (5,) (5,)
adding 439
[debug:559] nans -> diff: 0.0, variance: 0.0


/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:391: RuntimeWarning: invalid value encountered in true_divide
  result = np.abs(result)/np.sqrt(np.nansum(histogram*arr1, axis=self.groups_axis, keepdims=True))


[debug:587] [(5, 1000), (5, 1000)]
[debug:591] 1, 0.5222, 0.0
[debug:587] [(5, 1000), (5, 1000)]
[debug:591] 1, 0.5222, 0.0
[debug:546] Getting: <src.rcu_metrics.StratifiedUCE object at 0x7fccf5fcb850>
[debug:280] globalres? False
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:293] @get, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedENCE object at 0x7fccf5fcb790>
[debug:280] globalres? False
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:293] @get, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
[debug:280] globalres? True
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:293] @get, reskeys: dict_keys(['__0__', '__1__', '__all__'])
[debug:280] globalres? True
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:293] @get, reskeys: dict_keys(['__0__', '__1__', '__all__'])


In [10]:
res

P95     MeanH       Dens      Gini      Cover
471    uce        4.826015  6.130642  13.994750  8.169284  16.260691
       ause-uce   2.359191  2.973686   6.597004  3.914650   7.613376
       ence       0.000203  0.000186   0.000202  0.000203   0.000201
       ause-ence  0.000084  0.000078   0.000087  0.000087   0.000085
439    uce        1.859255  2.441899   5.539462  3.230504   6.134424
       ause-uce   1.768784  2.281610   5.099823  3.009890   5.743859
       ence       0.000140  0.000201   0.000199  0.000180   0.000194
       ause-ence  0.000066  0.000083   0.000086  0.000086   0.000083
global uce        5.168083  2.441899   0.007108  0.000926   0.010680
       ause-uce   4.910363  2.281610   0.005283  0.000686   0.008649
       ence       0.000195  0.000201   0.000111  0.000102   0.000089
       ause-ence  0.000084  0.000083   0.000034  0.000032   0.000028

### CI90 [pOK]

In [11]:
res, rcu = test(["ci90_accs"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:589] (5, 1000)
[debug:600] (5, 1000)
[debug:546] Getting: <src.rcu_metrics.StratifiedCIAccuracy object at 0x7fccf5ffe3d0>
[debug:213] getting [0]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.305 ((1, 1000))
[debug:227] @get, reskeys: dict_keys([])
[debug:239] @get_subset, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
adding 471


/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:31: RuntimeWarning: invalid value encountered in true_divide
  return np.nansum(values*counts, axis=axis, keepdims=keepdims)/np.nansum(counts, axis=axis, keepdims=keepdims)


[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:589] (5, 1000)
[debug:600] (5, 1000)
[debug:546] Getting: <src.rcu_metrics.StratifiedCIAccuracy object at 0x7fccf5ffe3d0>
[debug:213] getting [1]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.438 ((1, 1000))
[debug:227] @get, reskeys: dict_keys(['__0__'])
[debug:239] @get_subset, reskeys: dict_keys(['__0__', '__1__'])
[debug:624] (5,) (5,)
[debug:217] globalres? True
[debug:184] Arr[2] nan: 0.3715 ((2, 1000))
[debug:227] @get, reskeys: dict_keys(['__0__', '__1__', '__all__'])


In [12]:
res

P95     MeanH      Dens      Gini     Cover
439    ci90_accs       0.999993  0.999857  0.821685  0.852386  0.853149
       ause-ci90_accs  0.995993  0.995846  0.821412  0.847530  0.855276
471    ci90_accs       0.999852  0.999617  0.763954  0.808580  0.788883
       ause-ci90_accs  0.997848  0.997606  0.765335  0.807590  0.793129
global ci90_accs       0.999871  0.999649  0.771778  0.814517  0.797593
       ause-ci90_accs  0.997850  0.997624  0.770584  0.811998  0.802726

### AUCE [pOK]

In [13]:
res, rcu = test(["auce"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:589] (5, 1000, 100)
[debug:600] (5, 1000, 100)
[debug:546] Getting: <src.rcu_metrics.StratifiedAUCE object at 0x7fccf5fca090>
[debug:213] getting [0]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.305 ((1, 1000, 100))


/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:31: RuntimeWarning: invalid value encountered in true_divide
  return np.nansum(values*counts, axis=axis, keepdims=keepdims)/np.nansum(counts, axis=axis, keepdims=keepdims)


[debug:227] @get, reskeys: dict_keys([])
[debug:239] @get_subset, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
adding 471
[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:589] (5, 1000, 100)
[debug:600] (5, 1000, 100)
[debug:546] Getting: <src.rcu_metrics.StratifiedAUCE object at 0x7fccf5fca090>
[debug:213] getting [1]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.438 ((1, 1000, 100))
[debug:227] @get, reskeys: dict_keys(['__0__'])
[debug:239] @get_subset, reskeys: dict_keys(['__0__', '__1__'])
[debug:624] (5,) (5,)
[debug:217] globalres? True
[debug:184] Arr[2] nan: 0.3715 ((2, 1000, 100))
[debug:227] @get, reskeys: dict_keys(['__0__', '__1__', '__all__'])


In [14]:
res

P95     MeanH      Dens      Gini     Cover
439    auce       0.368993  0.317995  0.067401  0.036826  0.065870
       ause-auce  0.367661  0.316630  0.066651  0.036606  0.064691
471    auce       0.355436  0.298897  0.104811  0.070676  0.097014
       ause-auce  0.354704  0.298232  0.102536  0.069996  0.094326
global auce       0.357274  0.301485  0.099741  0.066089  0.092793
       ause-auce  0.356379  0.300839  0.099267  0.066479  0.089476

### C_v [pOK]

In [15]:
res, rcu = test(["cv"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:587] [(5, 1000), (5, 1000)]
[debug:591] 0, 0.5222, 0.0
[debug:546] Getting: <src.rcu_metrics.StratifiedCv object at 0x7fccf5fca8d0>
[debug:280] globalres? False
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:293] @get, reskeys: dict_keys([])
[debug:624] (5,) (5,)
adding 471


/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:31: RuntimeWarning: invalid value encountered in true_divide
  return np.nansum(values*counts, axis=axis, keepdims=keepdims)/np.nansum(counts, axis=axis, keepdims=keepdims)
/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:467: RuntimeWarning: invalid value encountered in true_divide
  result = np.sqrt(np.nansum(arr2, axis=axes, keepdims=True)/(np.nansum(histogram, axis=axes, keepdims=True)-1))/mu
/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:467: RuntimeWarning: invalid value encountered in sqrt
  result = np.sqrt(np.nansum(arr2, axis=axes, keepdims=True)/(np.nansum(histogram, axis=axes, keepdims=True)-1))/mu


[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:587] [(5, 1000), (5, 1000)]
[debug:591] 1, 0.5102, 0.0
[debug:546] Getting: <src.rcu_metrics.StratifiedCv object at 0x7fccf5fca8d0>
[debug:280] globalres? False
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:293] @get, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
[debug:280] globalres? True
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:293] @get, reskeys: dict_keys(['__0__', '__1__', '__all__'])


In [16]:
res

P95     MeanH      Dens      Gini     Cover
439    cv       0.006044  0.005152  0.003871  0.004707  0.004296
       ause-cv  0.008827  0.007887  0.007660  0.008074  0.006905
471    cv       0.009312  0.008257  0.005409  0.007123  0.005156
       ause-cv  0.008990  0.007995  0.006896  0.008058  0.006155
global cv       0.009032  0.008257  0.003010  0.004724  0.003838
       ause-cv  0.008559  0.007995  0.003157  0.004594  0.004731

### SRP [pOK]

In [17]:
res, rcu = test(["srp"], lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:589] (5, 1000)
[debug:600] (5, 1000)
[debug:546] Getting: <src.rcu_metrics.StratifiedSRP object at 0x7fccf5fcaa10>
[debug:213] getting [0]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.305 ((1, 1000))
[debug:227] @get, reskeys: dict_keys([])
[debug:239] @get_subset, reskeys: dict_keys(['__0__'])
[debug:624] (5,) (5,)
adding 471


/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:31: RuntimeWarning: invalid value encountered in true_divide
  return np.nansum(values*counts, axis=axis, keepdims=keepdims)/np.nansum(counts, axis=axis, keepdims=keepdims)


[debug:559] nans -> diff: 0.0, variance: 0.0
[debug:589] (5, 1000)
[debug:600] (5, 1000)
[debug:546] Getting: <src.rcu_metrics.StratifiedSRP object at 0x7fccf5fcaa10>
[debug:213] getting [1]
[debug:217] globalres? False
[debug:184] Arr[2] nan: 0.438 ((1, 1000))
[debug:227] @get, reskeys: dict_keys(['__0__'])
[debug:239] @get_subset, reskeys: dict_keys(['__0__', '__1__'])
[debug:624] (5,) (5,)
[debug:217] globalres? True
[debug:184] Arr[2] nan: 0.3715 ((2, 1000))
[debug:227] @get, reskeys: dict_keys(['__0__', '__1__', '__all__'])


In [18]:
res

P95     MeanH      Dens      Gini     Cover
439    srp       7.675559  4.135724  0.009542  0.001838  0.012452
       ause-srp  7.447388  3.965252  0.008634  0.001747  0.011087
471    srp       5.093514  2.487223  0.006748  0.001285  0.011395
       ause-srp  4.980209  2.408655  0.006345  0.001225  0.010625
global srp       5.443459  2.710645  0.007127  0.001360  0.011538
       ause-srp  5.564700  2.775924  0.006880  0.001349  0.010728

## Test RCU functionalities

In [ ]:
del res, rcu

### Results stability

In [ ]:
def test_stability(lo_variance, hi_variance, projects, cfg, N=2):
    print("creating")
    rcu = rcu = StratifiedRCU(
        num_variables=len(cfg["data_bands"]),
        # num_groups=len(projects),
        num_groups=N, # Debug
        num_bins=cfg["num_bins"],
        lo_variance=lo_variance,
        hi_variance=hi_variance
    )
    print("adding")
    results = {"orig": {}, "second": {}}
    for i, project in enumerate(projects):
        print("adding", project[0])
        rcu.add_project(*project)
        results["orig"][project[0]] = rcu.get_subset([project[0]])
    results["orig"]["global"] = rcu.get()
    for pid, _, _, _ in projects:
        results["second"][pid] = rcu.get_subset([pid])
    results["second"]["global"] = rcu.get()
    df1 = res2df(results["orig"], cfg)
    df2 = res2df(results["second"], cfg)
    df = pd.merge(df1, df2, left_index=True, right_index=True, suffixes=("_orig", "_second"))
    return results, df, rcu

In [ ]:
df[]

### Copy

In [35]:
def test_copy(lo_variance, hi_variance, projects, cfg, N=2):
    print("creating")
    rcu = rcu = StratifiedRCU(
        num_variables=len(cfg["data_bands"]),
        # num_groups=len(projects),
        num_groups=N, # Debug
        num_bins=cfg["num_bins"],
        lo_variance=lo_variance,
        hi_variance=hi_variance
    )
    print("adding")
    results = {"orig": {}, "copy": {}}
    for i, project in enumerate(projects):
        print("adding", project[0])
        rcu.add_project(*project)
        results["orig"][project[0]] = rcu.get_subset([project[0]])
    results["orig"]["global"] = rcu.get()
    rcu_c = rcu.copy()
    for pid, _, _, _ in projects:
        results["copy"][pid] = rcu_c.get_subset([pid])
    results["copy"]["global"] = rcu_c.get()
    df1 = res2df(results["orig"], cfg)
    df2 = res2df(results["copy"], cfg)
    df = pd.merge(df1, df2, left_index=True, right_index=True, suffixes=("_orig", "_copy"))
    return results, df, rcu, rcu_c

In [36]:
res, df, rcu1, rcu2 = test_copy(lo_variance, hi_variance, projects, cfg)

creating
adding
adding 439
[debug:546] Getting: <src.rcu_metrics.StratifiedMSE object at 0x7fccf71f5d10>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedRMSE object at 0x7fccf71f5510>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedMAE object at 0x7fccf71f52d0>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedMBE object at 0x7fccf71f5250>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedNLL object at 0x7fccf71f55d0>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedUCE object at 0x7fccf71f5890>
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedENCE object at 0x7fccf71f9b50>
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)


/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:31: RuntimeWarning: invalid value encountered in true_divide
  return np.nansum(values*counts, axis=axis, keepdims=keepdims)/np.nansum(counts, axis=axis, keepdims=keepdims)
/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:348: RuntimeWarning: invalid value encountered in true_divide
  return np.nanmean(0.5 * (np.log(variance) + (diff**2)/variance), axis=0)
/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:364: RuntimeWarning: invalid value encountered in true_divide
  elif variable=="diff": return np.nanmean(values**2, axis=0)
/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:369: RuntimeWarning: invalid value encountered in true_divide
  if arr2 is None: arr2 = self.X2.copy()


[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedCIAccuracy object at 0x7fccf71f9690>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedAUCE object at 0x7fccf71f9dd0>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedCv object at 0x7fccf71f5050>
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedSRP object at 0x7fccf71f9990>
[debug:624] values: (5,), ause: (5,)
adding 471
[debug:546] Getting: <src.rcu_metrics.StratifiedMSE object at 0x7fccf71f5d10>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedRMSE object at 0x7fccf71f5510>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedMAE object at 0x7fccf71f52d0>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedMBE object at 0x7fccf71f5250>
[debug:624] v

/scratch/ewalt/pdm/rs-uncertainty/src/rcu_metrics.py:440: RuntimeWarning: invalid value encountered in true_divide
  cerr = np.abs(self.rhos-empirical)


[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedNLL object at 0x7fccf71f9590>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedUCE object at 0x7fccf71f92d0>
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedENCE object at 0x7fccf71f9890>
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedCIAccuracy object at 0x7fccf71f90d0>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedAUCE object at 0x7fcd1ad8e810>
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedCv object at 0x7fccf71f96d0>
[Debug:218] cumX: (5, 1000), cumH: (5, 2, 1000)
[debug:624] values: (5,), ause: (5,)
[debug:546] Getting: <src.rcu_metrics.StratifiedSRP object at 0x7fccf71f9a90>
[debug:624] values: (5,), ause: (5,)
[Debug

In [37]:
df.iloc[:44]

P95_orig  MeanH_orig  Dens_orig  Gini_orig  Cover_orig  \
439 mse             0.325619    0.345869   0.014802   0.002589    0.019557   
    ause-mse        0.316299    0.332808   0.013426   0.002458    0.017563   
    rmse            0.546519    0.566615   0.115757   0.048444    0.127427   
    ause-rmse       0.537798    0.554638   0.110258   0.047084    0.120157   
    mae             0.435301    0.450296   0.092023   0.037767    0.101679   
    ause-mae        0.428070    0.440366   0.087426   0.036710    0.095359   
    mbe            -0.042112   -0.027191  -0.004988  -0.003574   -0.015277   
    ause-mbe       -0.042008   -0.026560  -0.003587  -0.003595   -0.013693   
    nll             0.962645    0.672108  -1.636143  -2.537406   -1.618837   
    ause-nll        0.946107    0.648937  -1.683600  -2.545449   -1.685208   
    uce             7.349941    3.789855   0.005351   0.000780    0.007276   
    ause-uce        7.131089    3.632444   0.004840   0.000729    0.006572   
    ence            0.001153    0.001303   0.000745   0.000462    0.000620   
    ause-ence       0.000511    0.000540   0.000292   0.000170    0.000187   
    ci90_accs       0.999993    0.999857   0.821685   0.852386    0.853149   
    ause-ci90_accs  0.995993    0.995846   0.821412   0.847530    0.855276   
    auce            0.368993    0.317995   0.067401   0.036826    0.065870   
    ause-auce       0.367661    0.316630   0.066651   0.036606    0.064691   
    cv              0.006044    0.004502   0.002207   0.003377    0.004216   
    ause-cv         0.004414    0.003382   0.001857   0.002510    0.003905   
    srp             7.675559    4.135724   0.009542   0.001838    0.012452   
    ause-srp        7.447388    3.965252   0.008634   0.001747    0.011087   
471 mse             0.267499    0.256655   0.014136   0.002227    0.022606   
    ause-mse        0.261826    0.248938   0.013323   0.002123    0.021195   
    rmse            0.492878    0.475826   0.113211   0.044448    0.142648   
    ause-rmse       0.487471    0.468161   0.109352   0.043481    0.136706   
    mae             0.387137    0.376022   0.089598   0.034760    0.113103   
    ause-mae        0.382698    0.369817   0.086444   0.034003    0.108049   
    mbe             0.033394    0.029016   0.000894   0.000052    0.006670   
    ause-mbe        0.032833    0.028638   0.000985   0.000020    0.006062   
    nll             0.742455    0.361909  -1.559775  -2.579001   -1.369920   
    ause-nll        0.731254    0.345214  -1.599821  -2.594327   -1.425450   
    uce             5.154241    2.430385   0.007685   0.001024    0.011087   
    ause-uce        5.270662    2.490887   0.007430   0.001021    0.010310   
    ence            0.000176    0.000170   0.000087   0.000102    0.000069   
    ause-ence       0.000128    0.000119   0.000056   0.000058    0.000046   
    ci90_accs       0.999852    0.999617   0.763954   0.808580    0.788883   
    ause-ci90_accs  0.997848    0.997606   0.765335   0.807590    0.793129   
    auce            0.355436    0.298897   0.104811   0.070676    0.097014   
    ause-auce       0.354704    0.298232   0.102536   0.069996    0.094326   
    cv              0.008629    0.007986   0.002862   0.004492    0.003512   
    ause-cv         0.008192    0.007750   0.003018   0.004382    0.004313   
    srp             5.093514    2.487223   0.006748   0.001285    0.011395   
    ause-srp        4.980209    2.408655   0.006345   0.001225    0.010625   

                    P95_copy  MeanH_copy  Dens_copy  Gini_copy  Cover_copy  
439 mse             0.325619    0.345869   0.014802   0.002589    0.019557  
    ause-mse        0.316299    0.332808   0.013426   0.002458    0.017563  
    rmse            0.546519    0.566615   0.115757   0.048444    0.127427  
    ause-rmse       0.537798    0.554638   0.110258   0.047084    0.120157  
    mae             0.435301    0.450296   0.092023   0.037767    0.101679  
    ause-mae        0.428070    0.440366   0.08

In [38]:
# no caching, no copies
df.iloc[44:]

P95_orig  MeanH_orig  Dens_orig  Gini_orig  Cover_orig  \
global mse             0.275376    0.268746   0.014226   0.002276    0.022193   
       ause-mse        0.283445    0.276293   0.013825   0.002284    0.020663   
       rmse            0.500148    0.488131   0.113556   0.044989    0.140585   
       ause-rmse       0.506596    0.494519   0.111485   0.045093    0.133984   
       mae             0.393665    0.386088   0.089927   0.035168    0.111555   
       ause-mae        0.398591    0.391233   0.088243   0.035224    0.105927   
       mbe             0.023160    0.021398   0.000097  -0.000439    0.003696   
       ause-mbe        0.025394    0.023076   0.000251  -0.000388    0.003514   
       nll             0.772297    0.403950  -1.570125  -2.573364   -1.403655   
       ause-nll        0.782579    0.417078  -1.588992  -2.563738   -1.468072   
       uce             5.168083    2.441899   0.007108   0.000926    0.010680   
       ause-uce        5.281255    2.499632   0.006949   0.000941    0.009956   
       ence            0.000195    0.000201   0.000111   0.000102    0.000089   
       ause-ence       0.000138    0.000134   0.000062   0.000056    0.000050   
       ci90_accs       0.999871    0.999649   0.771778   0.814517    0.797593   
       ause-ci90_accs  0.997850    0.997624   0.770584   0.811998    0.802726   
       auce            0.357274    0.301485   0.099741   0.066089    0.092793   
       ause-auce       0.356379    0.300839   0.099267   0.066479    0.089476   
       cv              0.009032    0.008257   0.003010   0.004724    0.003838   
       ause-cv         0.008559    0.007995   0.003157   0.004594    0.004731   
       srp             5.443459    2.710645   0.007127   0.001360    0.011538   
       ause-srp        5.564700    2.775924   0.006880   0.001349    0.010728   

                       P95_copy  MeanH_copy  Dens_copy  Gini_copy  Cover_copy  
global mse             0.275376    0.268746   0.014226   0.002276    0.022193  
       ause-mse        0.283445    0.276293   0.013825   0.002284    0.020663  
       rmse            0.500148    0.488131   0.113556   0.044989    0.140585  
       ause-rmse       0.506596    0.494519   0.111485   0.045093    0.133984  
       mae             0.393665    0.386088   0.089927   0.035168    0.111555  
       ause-mae        0.398591    0.391233   0.088243   0.035224    0.105927  
       mbe             0.023160    0.021398   0.000097  -0.000439    0.003696  
       ause-mbe        0.025394    0.023076   0.000251  -0.000388    0.003514  
       nll             0.772297    0.403950  -1.570125  -2.573364   -1.403655  
       ause-nll        0.782579    0.417078  -1.588992  -2.563738   -1.468072  
       uce             5.168083    2.441899   0.007108   0.000926    0.010680  
       ause-uce        5.281255    2.499632   0.006949   0.000941    0.009956  
       ence            0.000195    0.000201   0.000111   0.000102    0.000089  
       ause-ence       0.000138    0.000134   0.000062   0.000056    0.000050  
       ci90_accs       0.999871    0.999649   0.771778   0.814517    0.797593  
       ause-ci90_accs  0.997850    0.997624   0.770584   0.811998    0.802726  
       auce            0.357274    0.301485   0.099741   0.066089    0.092793  
       ause-auce       0.356379    0.300839   0.099267   0.066479    0.089476  
       cv              0.009032    0.008257   0.003010   0.004724    0.003838  
       ause-cv         0.008559    0.007995   0.003157   0.004594    0.004731  
       srp             5.443459    2.710645   0.007127   0.001360    0.011538  
       ause-srp        5.564700    2.775924   0.006880   0.001349    0.010728

### Upsample

In [ ]:
def test_upsample(rcu, ks):
    # visually: for a single project
    # plot all X or (X1, X2)
    pass